In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用生成式AI和BigQuery DataFrames进行代码生成

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/tree/main/notebooks/getting_started/bq_dataframes_llm_code_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/getting_started/bq_dataframes_llm_code_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/getting_started/bq_dataframes_llm_code_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

**_注意_**：此笔记本已在以下环境中测试：

* Python 版本 = 3.10

## 概述

使用这份笔记本来演示如何利用BigQuery DataFrames及其与Vertex AI上的生成式AI支持集成来生成样本代码的示例用例。

了解更多关于[BigQuery DataFrames](https://cloud.google.com/python/docs/reference/bigframes/latest)。

### 目标

在本教程中，您将创建一个包含调用给定一组API的示例代码的CSV文件。

步骤包括：

- 在BigQuery DataFrames中定义一个LLM模型，具体地说是PaLM API的[`text-bison`模型](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text)，使用`bigframes.ml.llm`。
- 通过从Cloud Storage中读取数据来创建一个DataFrame。
- 操纵DataFrame中的数据以构建LLM提示。
- 使用`predict`方法将DataFrame提示发送到LLM模型。
- 创建并使用一个自定义函数来转换LLM模型响应提供的输出。
- 将生成的转换后的DataFrame导出为CSV文件。

### 数据集

本教程使用了一个包含各种pandas DataFrame和Series API名称的数据集。

### 成本

本教程使用 Google Cloud 的计费组件：

* BigQuery
* Vertex AI 上的生成式 AI 支持
* Cloud Functions

了解 [BigQuery 计算定价](https://cloud.google.com/bigquery/pricing#analysis_pricing_models),
[Vertex AI 上的生成式 AI 支持定价](https://cloud.google.com/vertex-ai/pricing#generative_ai_models) 和 [Cloud Functions 定价](https://cloud.google.com/functions/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您预期的使用情况生成一个成本估算。

## 安装

安装以下包，这些包是运行这个笔记本所需要的：

In [ ]:
!pip install bigframes --upgrade --quiet

在开始之前

完成本节中的任务，以设置您的环境。

### 设置您的Google云项目

**以下步骤是必需的，无论您使用的笔记本环境是什么。**

1. [选择或创建一个Google云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您会获得$300的计算/存储成本抵扣。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [点击这里](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,cloudfunctions.googleapis.com,run.googleapis.com,artifactregistry.googleapis.com,cloudbuild.googleapis.com,cloudresourcemanager.googleapis.com) 启用以下API：

  * BigQuery API
  * BigQuery Connection API
  * Cloud Functions API
  * Cloud Run API
  * Artifact Registry API
  * Cloud Build API
  * Cloud Resource Manager API
  * Vertex AI API

4. 如果您正在本地运行这个笔记本，请安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

如果您不知道您的项目ID，请尝试以下方法：
* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

设置区域

您也可以更改BigQuery使用的`REGION`变量。了解有关[BigQuery区域](https://cloud.google.com/bigquery/docs/locations#supported_locations)的更多信息。

In [ ]:
REGION = "US"  # @param {type: "string"}

### 验证您的谷歌云帐户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关指示操作。

**顶点 AI 工作台**

不用做任何事，您已经通过验证了。

本地JupyterLab实例

取消注释并运行以下单元格：

In [ ]:
# ! gcloud auth login

取消注释并运行以下单元格:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### 导入库

In [ ]:
import bigframes.pandas as bf
from google.cloud import bigquery_connection_v1 as bq_connection

### 设置 BigQuery 数据帧选项

In [ ]:
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

如果您想要重置已创建的DataFrame或Series对象的位置，请执行`bf.reset_session()`来重置会话。之后，您可以重复使用`bf.options.bigquery.location`来指定另一个位置。

# 定义LLM模型

BigQuery DataFrames 通过 Vertex AI 与 PaLM API 的 `text-bison` 模型集成。

本节将介绍在笔记本中使用该模型所需的一些步骤。

创建一个BigQuery云资源连接

您需要创建一个[云资源连接](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection)，以使BigQuery DataFrames能够与Vertex AI服务进行交互。

In [ ]:
CONN_NAME = "bqdf-llm"

client = bq_connection.ConnectionServiceClient()
new_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}"
exists_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
cloud_resource_properties = bq_connection.CloudResourceProperties({})

try:
    request = client.get_connection(
        request=bq_connection.GetConnectionRequest(name=exists_conn_parent)
    )
    CONN_SERVICE_ACCOUNT = f"serviceAccount:{request.cloud_resource.service_account_id}"
except Exception:
    connection = bq_connection.types.Connection(
        {"friendly_name": CONN_NAME, "cloud_resource": cloud_resource_properties}
    )
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": new_conn_parent,
            "connection_id": CONN_NAME,
            "connection": connection,
        }
    )
    response = client.create_connection(request)
    CONN_SERVICE_ACCOUNT = (
        f"serviceAccount:{response.cloud_resource.service_account_id}"
    )
print(CONN_SERVICE_ACCOUNT)

## 为服务账号设置权限

资源连接服务账号需要特定的项目级权限：
 - `roles/aiplatform.user` 和 `roles/bigquery.connectionUser`: 这些角色是连接到 Vertex AI 中使用 LLM 模型创建模型定义所需的权限（[文档](https://cloud.google.com/bigquery/docs/generate-text#give_the_service_account_access)）。
 - `roles/run.invoker`: 这个角色是连接拥有对支持自定义/远程函数的 Cloud Run 服务的只读访问权限所需的权限（[文档](https://cloud.google.com/bigquery/docs/remote-functions#grant_permission_on_function)）。

通过运行以下 `gcloud` 命令设置这些权限：

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/aiplatform.user'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/run.invoker'

## 定义模型

使用`bigframes.ml.llm`来定义模型：

In [ ]:
from bigframes.ml.llm import PaLM2TextGenerator

session = bf.get_global_session()
connection = f"{PROJECT_ID}.{REGION}.{CONN_NAME}"
model = PaLM2TextGenerator(session=session, connection_name=connection)

从Cloud Storage读取数据到BigQuery DataFrames

您可以通过从以下任何位置读取数据来创建BigQuery DataFrames DataFrame：

* 本地数据文件
* 存储在BigQuery表中的数据
* 存储在Cloud Storage中的数据文件
* 内存中的pandas DataFrame

在本教程中，您将通过从Cloud Storage中存储的两个CSV文件读取数据来创建BigQuery DataFrames DataFrames，其中一个包含DataFrame API名称列表，另一个包含Series API名称列表。

In [ ]:
df_api = bf.read_csv("gs://cloud-samples-data/vertex-ai/bigframe/df.csv")
series_api = bf.read_csv("gs://cloud-samples-data/vertex-ai/bigframe/series.csv")

查看每个文件的几行数据。

In [ ]:
df_api.head(2)

In [ ]:
series_api.head(2)

使用LLM模型生成代码

准备提示并将其发送给LLM模型进行预测。

## 在BigQuery DataFrames中设计提示

为LLMs设计提示是一个快速发展的领域，您可以在[此文档](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/introduction-prompt-design)中阅读更多。

在本教程中，您将使用一个简单的提示来询问LLM模型获取上一步DataFrames中每个API方法（或行）的示例代码。输出是新的DataFrames `df_prompt` 和 `series_prompt`，其中包含完整的提示文本。

In [ ]:
df_prompt_prefix = "Generate Pandas sample code for DataFrame."
series_prompt_prefix = "Generate Pandas sample code for Series."

df_prompt = df_prompt_prefix + df_api["API"]
series_prompt = series_prompt_prefix + series_api["API"]

df_prompt.head(2)

使用LLM模型进行预测

使用包含完整提示文本的BigQuery DataFrames DataFrame作为`predict`方法的输入。`predict`方法调用LLM模型，并将生成的文本输出返回到两个新的BigQuery DataFrames DataFrames，`df_pred`和`series_pred`。

注意：预测可能需要几分钟才能运行。

In [ ]:
df_pred = model.predict(df_prompt.to_frame(), max_output_tokens=1024)
series_pred = model.predict(series_prompt.to_frame(), max_output_tokens=1024)

一旦预测被处理，查看来自LLM的示例输出，该输出为数据框数据集中列出的API名称提供了代码示例。

In [ ]:
print(df_pred["ml_generate_text_llm_result"].iloc[0])

使用远程函数操纵LLM输出

LLM提供的输出通常包含代码示例本身之外的其他文本。使用BigQuery数据帧，您可以部署定制的Python函数来处理和转换这个输出。

运行下面的单元格将创建一个自定义函数，您可以使用它以两种方式处理LLM输出数据：
1. 剥离LLM文本输出，只包括代码块。
2. 将`import pandas as pd`替换为`import bigframes.pandas as bf`，以便生成的代码块与BigQuery数据框架一起使用。

In [ ]:
@bf.remote_function([str], str, bigquery_connection=CONN_NAME)
def extract_code(text: str):
    try:
        res = text[text.find("\n") + 1 : text.find("```", 3)]
        res = res.replace("import pandas as pd", "import bigframes.pandas as bf")
        if "import bigframes.pandas as bf" not in res:
            res = "import bigframes.pandas as bf\n" + res
            return res
    except:
        return ""

该自定义函数部署为云函数，然后与 BigQuery 集成为[远程函数](https://cloud.google.com/bigquery/docs/remote-functions)。保存这两个函数的名称，以便在本笔记本的末尾清理它们。

In [ ]:
CLOUD_FUNCTION_NAME = format(extract_code.bigframes_cloud_function)
print("Cloud Function Name " + CLOUD_FUNCTION_NAME)
REMOTE_FUNCTION_NAME = format(extract_code.bigframes_remote_function)
print("Remote Function Name " + REMOTE_FUNCTION_NAME)

将自定义函数应用于每个LLM输出DataFrame，以获取处理后的结果：

In [ ]:
df_code = df_pred.assign(
    code=df_pred["ml_generate_text_llm_result"].apply(extract_code)
)
series_code = series_pred.assign(
    code=series_pred["ml_generate_text_llm_result"].apply(extract_code)
)

您可以通过检查数据的第一行来观察到差异。

In [ ]:
print(df_code["code"].iloc[0])

将结果保存到云存储

BigQuery DataFrames可以让您将BigQuery DataFrames数据帧保存为CSV文件，以便进一步在云存储中使用。现在可以尝试使用您处理过的LLM输出数据进行这个操作。

创建一个具有独特名称的新云存储桶：

In [ ]:
import uuid

BUCKET_ID = "code-samples-" + str(uuid.uuid1())

!gsutil mb gs://{BUCKET_ID}

使用`to_csv`将每个BigQuery DataFrames DataFrame写入Cloud Storage存储桶中的CSV文件：

In [ ]:
df_code[["code"]].to_csv(f"gs://{BUCKET_ID}/df_code*.csv")
series_code[["code"]].to_csv(f"gs://{BUCKET_ID}/series_code*.csv")

您可以导航到Cloud Storage存储桶浏览器下载这两个文件并查看它们。

运行以下单元格，然后按照链接导航到您的Cloud Storage存储桶浏览器：

In [ ]:
print(f"https://console.developers.google.com/storage/browser/{BUCKET_ID}/")

摘要和下一步

您已经使用了BigQuery DataFrames与LLM模型(`bigframes.ml.llm`)的集成来生成代码示例，并通过在BigQuery DataFrames中创建和使用自定义函数来转换LLM输出。

在[文档](https://cloud.google.com/python/docs/reference/bigframes/latest)中了解更多关于BigQuery DataFrames的信息，并在[GitHub仓库](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks)中找到更多示例笔记本。

清理

要清理此项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以取消注释剩余的单元格并运行它们以删除您在本教程中创建的各个资源。

In [ ]:
# # Delete the BigQuery Connection
# from google.cloud import bigquery_connection_v1 as bq_connection
# client = bq_connection.ConnectionServiceClient()
# CONNECTION_ID = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
# client.delete_connection(name=CONNECTION_ID)
# print(f"Deleted connection '{CONNECTION_ID}'.")

In [ ]:
# # Delete the Cloud Function
# ! gcloud functions delete {CLOUD_FUNCTION_NAME} --quiet
# # Delete the Remote Function
# REMOTE_FUNCTION_NAME = REMOTE_FUNCTION_NAME.replace(PROJECT_ID + ".", "")
# ! bq rm --routine --force=true {REMOTE_FUNCTION_NAME}

In [ ]:
# # Delete the Google Cloud Storage bucket and files
# ! gsutil rm -r gs://{BUCKET_ID}
# print(f"Deleted bucket '{BUCKET_ID}'.")